In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pandas as pd
import os

In [4]:
deprel = "discourse"

In [5]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-11-at-2023-06-02-19-35-84227adb.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [6]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_discourse_export_big_v1_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [7]:
data[0]

{'id': 71810,
 'annotations': [{'id': 22173,
   'completed_by': 1,
   'result': [{'value': {'start': 2,
      'end': 6,
      'text': 'Kõss',
      'labels': ['discourse_phrases']},
     'id': '_C9-7AKNXR',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['bound']},
     'id': 'mJ5BaHOk10',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-06-02T18:11:46.741122Z',
   'updated_at': '2023-06-02T18:11:46.741157Z',
   'lead_time': 7.945,
   'prediction': {'id': 71809,
    'model_version': 'undefined',
    'created_ago': '0\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 2,
       'end': 6,
       'text': ['Kõss'],
       'labels': ['discourse_phrases']}}],
    'score': None,
    'cluster': None,
    'neighb

In [8]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [9]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [10]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,71810,Train/ilu_valton_ud211.conllu,""" Kõss , kana ! """,2,6,Kõss,bound,""" , kana ! """,100.0,100.0,100.0
1,71811,Train/ilu_vilep_ud211.conllu,""" Ema , ütle mulle mu aadress , palun !",30,37,", palun",free,""" Ema , ütle mulle mu aadress !",100.0,100.0,100.0
2,71812,Train/ilu_kivirahk_ud211.conllu,No ja veel igasuguseid raamtuid - Lenini teose...,0,2,No,free,ja veel igasuguseid raamtuid - Lenini teoseid ...,94.1,94.1,100.0
3,71813,Train/aja_ee200110_osa_7_ud211.conllu,""" Oi , neid kirsina punavaid huuli , "" laulab ...",2,4,Oi,bound,""" , neid kirsina punavaid huuli , "" laulab ta ...",95.7,95.7,95.7
4,71814,Train/ilu_vilep_ud211.conllu,""" Kurat , "" Postimeest "" siis veel polnud ju !",0,9,""" Kurat ,",free,""" Postimeest "" siis veel polnud ju !",100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
249,72059,Train/ilu_kanep_ud211.conllu,""" Ahah , et isa on tööl , noh eks ma tulen mõn...",2,6,Ahah,redundant comma,""" , et isa on tööl , noh eks ma tulen mõnipäev...",93.3,100.0,93.3
250,72060,Train/ilu_kanep_ud211.conllu,""" Ahah , et isa on tööl , noh eks ma tulen mõn...",26,29,noh,free,""" Ahah , et isa on tööl , eks ma tulen mõnipäe...",66.7,73.3,80.0
251,72061,Train/aja_ee200110_osa_1_ud211.conllu,Mitmed vastused ajakirjanike küsimustele lõpet...,99,108,""" aitäh """,bound,Mitmed vastused ajakirjanike küsimustele lõpet...,100.0,100.0,100.0
252,72062,Train/ilu_indrikson_ud211.conllu,"Siin oli kõik teisiti , isegi kliima oli kehve...",48,56,", ausalt",free,"Siin oli kõik teisiti , isegi kliima oli kehvem !",90.0,90.0,100.0


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,71811,Train/ilu_vilep_ud211.conllu,""" Ema , ütle mulle mu aadress , palun !",30,37,", palun",free,""" Ema , ütle mulle mu aadress !",100.0,100.0,100.0
2,71812,Train/ilu_kivirahk_ud211.conllu,No ja veel igasuguseid raamtuid - Lenini teose...,0,2,No,free,ja veel igasuguseid raamtuid - Lenini teoseid ...,94.1,94.1,100.0
4,71814,Train/ilu_vilep_ud211.conllu,""" Kurat , "" Postimeest "" siis veel polnud ju !",0,9,""" Kurat ,",free,""" Postimeest "" siis veel polnud ju !",100.0,100.0,100.0
11,71821,Train/ilu_kivirahk_ud211.conllu,""" No siis sa oled ju kõvasti vaeva näinud , "" ...",2,4,No,free,""" siis sa oled ju kõvasti vaeva näinud , "" ütl...",76.9,76.9,84.6
13,71823,Train/ilu_kivirahk_ud211.conllu,""" Ah soo ! "" naeris Ernesaks mürgiselt .",2,4,Ah,free,""" soo ! "" naeris Ernesaks mürgiselt .",100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
246,72056,Train/ilu_vilep_ud211.conllu,No me veel vaatame seda asja !,0,2,No,free,me veel vaatame seda asja !,100.0,100.0,100.0
247,72057,Train/ilu_kanep_ud211.conllu,"Ma ei pidanud seda siin silmas , ausalt , noh !",40,45,", noh",free,"Ma ei pidanud seda siin silmas , ausalt !",11.1,11.1,44.4
250,72060,Train/ilu_kanep_ud211.conllu,""" Ahah , et isa on tööl , noh eks ma tulen mõn...",26,29,noh,free,""" Ahah , et isa on tööl , eks ma tulen mõnipäe...",66.7,73.3,80.0
252,72062,Train/ilu_indrikson_ud211.conllu,"Siin oli kõik teisiti , isegi kliima oli kehve...",48,56,", ausalt",free,"Siin oli kõik teisiti , isegi kliima oli kehvem !",90.0,90.0,100.0


In [12]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [13]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
#df_free_cons

In [14]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [11]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,71810,Train/ilu_valton_ud211.conllu,""" Kõss , kana ! """,2,6,Kõss,bound,""" , kana ! """,100.0,100.0,100.0
3,71813,Train/aja_ee200110_osa_7_ud211.conllu,""" Oi , neid kirsina punavaid huuli , "" laulab ...",2,4,Oi,bound,""" , neid kirsina punavaid huuli , "" laulab ta ...",95.7,95.7,95.7
5,71815,Train/ilu_vilep_ud211.conllu,""" Issand , "" tuli nutuvõre ümber naise suu , ""...",0,12,""" Issand , """,bound,"tuli nutuvõre ümber naise suu , "" ma arvasin ,...",80.0,80.0,86.7
16,71826,Train/ilu_indrikson_ud211.conllu,""" Ahh ? "" jahmus Indrikson .",2,5,Ahh,bound,""" ? "" jahmus Indrikson .",100.0,100.0,100.0
17,71827,Train/ilu_maailm_ud211.conllu,""" Aitäh , "" naeratas Konspiraator oma hambutut...",0,11,""" Aitäh , """,bound,"naeratas Konspiraator oma hambutut naeratust ,...",91.7,91.7,100.0
19,71829,Train/ilu_kanep_ud211.conllu,"Hommikust Eesti , Soome ja Setumaa .",0,9,Hommikust,bound,"Eesti , Soome ja Setumaa .",83.3,83.3,100.0
22,71832,Train/ilu_kanep_ud211.conllu,""" Ah , "" ütles Merike , kui tema suu hetkeks v...",0,8,""" Ah , """,bound,"ütles Merike , kui tema suu hetkeks vabanes .",88.9,88.9,88.9
29,71839,Train/ilu_kanep_ud211.conllu,Krapsti ! olen laua juures .,0,7,Krapsti,bound,! olen laua juures .,80.0,80.0,80.0
33,71843,Train/ilu_kanep_ud211.conllu,""" Nojah , "" jäin minagi mõtlema .",2,7,Nojah,bound,""" , "" jäin minagi mõtlema .",100.0,100.0,100.0
37,71847,Train/ilu_vilep_ud211.conllu,Einoh -- mõtles Jan -- kui nii siis nii !,0,5,Einoh,bound,-- mõtles Jan -- kui nii siis nii !,77.8,77.8,100.0


In [12]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [13]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
6,71816,Train/ilu_kanep_ud211.conllu,""" Oi , sorry , ma ei tea , mis mulle sisse läk...",2,4,Oi,redundant comma,""" , sorry , ma ei tea , mis mulle sisse läks ,...",84.2,84.2,89.5
7,71817,Train/ilu_kanep_ud211.conllu,""" Oi , sorry , ma ei tea , mis mulle sisse läk...",7,12,sorry,redundant comma,""" Oi , , ma ei tea , mis mulle sisse läks , "" ...",84.2,84.2,89.5
8,71818,Train/ilu_kanep_ud211.conllu,"Asjad on ju seotud ja selged , muuseas , as-ja...",52,55,jah,redundant comma,"Asjad on ju seotud ja selged , muuseas , as-ja...",50.0,62.5,75.0
9,71819,Train/ilu_kanep_ud211.conllu,"Asjad on ju seotud ja selged , muuseas , as-ja...",66,69,jaa,redundant comma,"Asjad on ju seotud ja selged , muuseas , as-ja...",43.8,68.8,68.8
10,71820,Train/ilu_kivirahk_ud211.conllu,""" Noh , eks need ole need niinimetatud tõekuul...",2,5,Noh,redundant comma,""" , eks need ole need niinimetatud tõekuulutaj...",73.3,80.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...
240,72050,Train/ilu_kanep_ud211.conllu,"Niisiis , raisk , ei mingeid piire enam , rais...",10,15,raisk,redundant comma,"Niisiis , , ei mingeid piire enam , raisk , ei...",85.7,85.7,100.0
244,72054,Train/ilu_vilep_ud211.conllu,""" Oi , andke andeks !",2,4,Oi,redundant comma,""" , andke andeks !",100.0,100.0,100.0
245,72055,Train/ilu_vilep_ud211.conllu,"Ah , et ei saa minna sinna , kuhu sa ise tahad ?",0,2,Ah,redundant comma,", et ei saa minna sinna , kuhu sa ise tahad ?",91.7,91.7,100.0
248,72058,Train/ilu_kanep_ud211.conllu,""" Nojah , ega mulgi viga ole """,2,7,Nojah,redundant comma,""" , ega mulgi viga ole """,100.0,100.0,100.0


In [14]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [15]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
12,71822,Train/ilu_kivirahk_ud211.conllu,""" Ah soo ! "" naeris Ernesaks mürgiselt .",0,12,""" Ah soo ! """,unnatural,naeris Ernesaks mürgiselt .,100.0,100.0,100.0
28,71838,Train/ilu_kivirahk_ud211.conllu,""" Oh , taadike ! "" vangutas Petrov pead .",0,18,""" Oh , taadike ! """,unnatural,vangutas Petrov pead .,100.0,100.0,100.0
143,71953,Train/ilu_vilep_ud211.conllu,""" Palun ! "" tegi Jan anuva grimassi .",0,11,""" Palun ! """,unnatural,tegi Jan anuva grimassi .,100.0,100.0,100.0
184,71994,Train/ilu_vilep_ud211.conllu,""" Tohoh ! "" ei saanud Jan enam millestki aru .",0,11,""" Tohoh ! """,unnatural,ei saanud Jan enam millestki aru .,100.0,100.0,100.0
205,72015,Train/ilu_vilep_ud211.conllu,"Jeeee ! "" kraaksatas ta täiesti õnnelikult .",0,9,"Jeeee ! """,unnatural,kraaksatas ta täiesti õnnelikult .,100.0,100.0,100.0
229,72039,Train/ilu_vilep_ud211.conllu,""" Mm ! "" püüdis ta kuidagi reageerida ja neela...",0,8,""" Mm ! """,unnatural,püüdis ta kuidagi reageerida ja neelas suutäie...,100.0,100.0,100.0


In [16]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [17]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [18]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [19]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
26,71836,Train/aja_ml200247_osa_2_ud211.conllu,"Ja ennäe - ei tulnudki kaua oodata , kui autog...",0,8,Ja ennäe,other redundant punctuation,"- ei tulnudki kaua oodata , kui autoga sõitis ...",81.8,81.8,93.9
40,71850,Train/aja_ee200110_osa_9_ud211.conllu,"Uih - ta teeb , nagu topiks mikrofoni tagumikku !",0,3,Uih,other redundant punctuation,"- ta teeb , nagu topiks mikrofoni tagumikku !",100.0,100.0,100.0
62,71872,Train/aja_ee199920_osa_3_ud211.conllu,Muide -- ka enesekeskne filmitegemine võib oll...,0,5,Muide,other redundant punctuation,-- ka enesekeskne filmitegemine võib olla sage...,94.1,94.1,100.0
84,71894,Dev/ilu_ruben_ud211.conllu,Ahhaa - sellist esildist teie ei ole teinud !,0,5,Ahhaa,other redundant punctuation,- sellist esildist teie ei ole teinud !,87.5,100.0,87.5
153,71963,Train/aja_ee200110_osa_9_ud211.conllu,"Ja oi , ai , ui - kas ta võtab nüüd püksid täi...",0,15,"Ja oi , ai , ui",other redundant punctuation,- kas ta võtab nüüd püksid täitsa maha või ...,90.0,100.0,90.0
224,72034,Train/ilu_kanep_ud211.conllu,Õhus heljus kõdunemise lehk ja ennäe - kerget ...,31,36,ennäe,other redundant punctuation,Õhus heljus kõdunemise lehk ja - kerget uduvin...,92.3,100.0,92.3


In [20]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:25:10.810369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:25:11.445975: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:25:13.151761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:25:13.151960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [16]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data



98it [00:10,  9.31it/s]


In [17]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")